
# **AntiFold for nanobody CDRs (re)design**
## Overview
####[Antifold](https://github.com/oxpig/AntiFold) ([paper link](https://academic.oup.com/bioinformaticsadvances/article/5/1/vbae202/8090019)) is a model trained to reconstruct the sequence of complementarity-determining regions (CDRs) of antibody- or nanobody-antigen complexes, so it resulted in a superior accuracy in the sequence (re)design of these regions with respect to ProteinMPNN, that was trained on a very tiny fraction of loop-mediated interactions
---



In [ ]:
# @title Install AntiFold and Dependencies
# @markdown Run this cell first to install all required packages

# Accept Conda Terms of Service first
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Initialize conda
!source /usr/local/bin/activate
!conda init bash

# Set up conda environment
!conda create --name antifold python=3.10 -y
# Use the full path to the conda environment to avoid activation issues
!/usr/local/envs/antifold/bin/pip install torch==2.2.0

# Clone and install AntiFold
!git clone https://github.com/oxpig/AntiFold
%cd AntiFold
!/usr/local/envs/antifold/bin/pip install .

# Install additional dependencies for the notebook
!pip install biopython pandas ipywidgets

# Create necessary directories
!mkdir -p data/pdbs
!mkdir -p output

print("Installation completed successfully!")

In [ ]:
# @title Run AntiFold

# @markdown **Run this cell to upload your PDB file, set parameters and run AntiFold for CDR redesign**

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import subprocess
from google.colab import widgets as colab_widgets # Import colab widgets

# Create form for parameter input
display(HTML("<h2>Nanobody Design Parameters</h2>"))

# Use Google Colab style widgets
nanobody_chain = widgets.Text(description="Nanobody Chain:", value="A")
antigen_chain = widgets.Text(description="Antigen Chain:", value="B")
num_sequences = widgets.IntSlider(description="Number of sequences:", min=1, max=100, value=10)
seed_value = widgets.Text(description="Random seed:", value="5748") # Keep as text for flexibility
temperature = widgets.FloatSlider(description="Sampling temperature:", min=0.1, max=1.0, step=0.1, value=0.3)

# Display the widgets
display(nanobody_chain)
display(antigen_chain)
display(num_sequences)
display(seed_value)
display(temperature)


# File upload section
display(HTML("<h3>Upload PDB File</h3>"))

upload_button = widgets.FileUpload(
    description="Choose PDB file",
    accept='.pdb',
    multiple=False
)
display(upload_button)

# Run button
run_button = widgets.Button(description="Run AntiFold", button_style='success')
output_area = widgets.Output()

def on_run_button_clicked(b):
    with output_area:
        clear_output()

        # Check if file was uploaded
        if not upload_button.value:
            print("Please upload a PDB file first.")
            return

        # Get uploaded file
        uploaded_file = list(upload_button.value.values())[0]
        file_name = list(upload_button.value.keys())[0]
        pdb_path = f"data/pdbs/{file_name}"

        # Save uploaded file
        with open(pdb_path, 'wb') as f:
            f.write(uploaded_file['content'])

        print(f"File {file_name} uploaded successfully.")

        # Extract antigen sequence using BioPython
        try:
            from Bio.PDB import PDBParser
            from Bio.SeqUtils import seq1

            parser = PDBParser()
            structure = parser.get_structure('nanobody_antigen', pdb_path)
            antigen_sequence = ""

            # Try to get antigen chain
            for model in structure:
                if antigen_chain.value in model:
                    chain = model[antigen_chain.value]
                    for residue in chain:
                        if residue.get_id()[0] == ' ':  # Exclude heteroatoms/water
                            try:
                                antigen_sequence += seq1(residue.get_resname())
                            except:
                                pass
            print(f"Antigen chain {antigen_chain.value} sequence: {antigen_sequence}")
        except Exception as e:
            print(f"Could not extract antigen sequence: {str(e)}")
            antigen_sequence = ""

        # Run AntiFold command using the full path to the conda environment's python
        cmd = [
            "/usr/local/envs/antifold/bin/python", "antifold/main.py",
            "--pdb_file", pdb_path,
            "--nanobody_chain", nanobody_chain.value,
            "--antigen_chain", antigen_chain.value,
            "--num_seq_per_target", str(num_sequences.value),
            "--seed", str(seed_value.value),
            "--sampling_temp", str(temperature.value),
            "--out_dir", "output"
        ]

        print(f"Running command: {' '.join(cmd)}")

        try:
            # Run the command
            result = subprocess.run(cmd, capture_output=True, text=True, cwd=".")

            # Print output
            print("STDOUT:")
            print(result.stdout)

            if result.stderr:
                print("STDERR:")
                print(result.stderr)

            if result.returncode == 0:
                print("AntiFold completed successfully!")

                # Save antigen sequence to file
                with open("output/antigen_sequence.txt", "w") as f:
                    f.write(antigen_sequence)

            else:
                print(f"AntiFold failed with return code {result.returncode}")

        except Exception as e:
            print(f"Error running AntiFold: {str(e)}")

run_button.on_click(on_run_button_clicked)
display(run_button)
display(output_area)

In [ ]:
# @title Results and Download
# @markdown Review and download the designed sequences

import pandas as pd
import os
from IPython.display import display, HTML, FileLink
from google.colab import files

# Check for output files
output_files = []
if os.path.exists("output"):
    output_files = os.listdir("output")

# Find the main output file (assuming CSV format)
csv_files = [f for f in output_files if f.endswith('.csv')]

if csv_files:
    # Display the first CSV file found
    csv_path = f"output/{csv_files[0]}"

    try:
        # Read and display the CSV file
        df = pd.read_csv(csv_path)
        display(HTML("<h3>Designed Sequences</h3>"))
        display(df)

        # Provide download link
        display(HTML("<h3>Download Results</h3>"))
        display(FileLink(csv_path, result_html_prefix="Download CSV: "))

        # Check if antigen sequence file exists
        antigen_file = "output/antigen_sequence.txt"
        if os.path.exists(antigen_file):
            with open(antigen_file, "r") as f:
                antigen_sequence = f.read()
            display(HTML(f"<h3>Antigen Sequence (Chain {antigen_chain.value})</h3>"))
            display(HTML(f"<p>{antigen_sequence}</p>"))
            display(FileLink(antigen_file, result_html_prefix="Download Antigen Sequence: "))

    except Exception as e:
        display(HTML(f"<p>Error reading output file: {str(e)}</p>"))
else:
    display(HTML("<p>No results yet. Please run AntiFold first.</p>"))

# Also check for FASTA files (if sequence sampling was performed)
fasta_files = [f for f in output_files if f.endswith('.fasta')]
for fasta_file in fasta_files:
    fasta_path = f"output/{fasta_file}"
    display(FileLink(fasta_path, result_html_prefix="Download FASTA: "))